In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

df = pd.read_excel("C:\\Users\\Mehmet Ali\\Desktop\\Datalar\\Pusula Academy\\Talent_Academy_Case_DT_2025.xlsx")

In [ ]:
# 
df_clean = df.copy()

In [ ]:


df_clean['Cinsiyet'] = df_clean['Cinsiyet'].fillna('Eksik')
df_clean['Alerji'] = df_clean['Alerji'].fillna('Bilinmiyor')
df_clean['KanGrubu'] = df_clean['KanGrubu'].fillna('Bilinmiyor')
df_clean['KronikHastalik'] = df_clean['KronikHastalik'].fillna('Bilinmiyor')
df_clean['UygulamaYerleri'] = df_clean['UygulamaYerleri'].fillna('Bilinmiyor')
df_clean['Tanilar'] = df_clean['Tanilar'].fillna('Bilinmiyor')
df_clean['Bolum'] = df_clean['Bolum'].fillna('Bilinmiyor')



df_clean['Yas'] = df_clean['Yas'].fillna(np.nan)

In [ ]:


df_clean['TedaviSuresi_Num'] = df_clean['TedaviSuresi'].str.extract('(\d+)').astype(float)
df_clean['UygulamaSuresi_Num'] = df_clean['UygulamaSuresi'].str.extract('(\d+)').astype(float)

In [ ]:


df_patient = df_clean.groupby('HastaNo').first().reset_index()



In [ ]:


agg_dict = {
    'Yas': 'first',                       
    'Cinsiyet': 'first',                   
    'KanGrubu': 'first',                  
    'Uyruk': 'first',                     
    'KronikHastalik': lambda x: ','.join(set(x.dropna().astype(str))),  
    'Bolum': lambda x: ','.join(set(x.dropna().astype(str))),
    'Alerji': lambda x: ','.join(set(x.dropna().astype(str))),
    'Tanilar': lambda x: ','.join(set(x.dropna().astype(str))),
    'TedaviAdi': lambda x: ','.join(set(x.dropna().astype(str))),
    'UygulamaYerleri': lambda x: ','.join(set(x.dropna().astype(str))),
    'UygulamaSuresi_Num': 'mean',          
    'TedaviSuresi_Num': 'mean'             
}

df_patient = df_clean.groupby('HastaNo').agg(agg_dict).reset_index()

print("Hasta bazlı veri boyutu:", df_patient.shape)


Hasta bazlı veri boyutu: (404, 13)


In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


target = "TedaviSuresi_Num"


features = ['Yas', 'Cinsiyet', 'KanGrubu', 'Uyruk', 
            'KronikHastalik', 'Bolum', 'Alerji', 
            'Tanilar', 'TedaviAdi', 'UygulamaYerleri', 
            'UygulamaSuresi_Num']

X = df_patient[features]
y = df_patient[target]


numeric_features = ['Yas', 'UygulamaSuresi_Num']
categorical_features = ['Cinsiyet', 'KanGrubu', 'Uyruk', 'KronikHastalik',
                        'Bolum', 'Alerji', 'Tanilar', 'TedaviAdi', 'UygulamaYerleri']


numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])


categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


pipeline = Pipeline(steps=[("preprocessor", preprocessor)])


X_train_processed = pipeline.fit_transform(X_train)
X_test_processed = pipeline.transform(X_test)

print("Ön işlenmiş train veri boyutu:", X_train_processed.shape)
print("Ön işlenmiş test veri boyutu:", X_test_processed.shape)


Ön işlenmiş train veri boyutu: (323, 733)
Ön işlenmiş test veri boyutu: (81, 733)
